<h2><b>Import</b></h2>

 Vì dữ liệu thu thập của chúng ta sẽ rất lớn, nên dùng thư viện scrapy để thu thập dữ liệu thêm về dân số một cách nhanh chóng.

In [5]:
!pip install scrapy
!pip install spider3
!pip install pandas
!pip install shutil
!pip install numpy


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import json
import re
import pandas as pd
import scrapy
import shutil
import os
import numpy as np
from pandas.testing import assert_frame_equal # to compare two dataframes
# YOUR CODE HERE (OPTION) 
# Nếu cần các thư viện khác thì bạn có thể import thêm tại đây

ImportError: cannot import name hashtable

     Dân số thế giới chính thức đạt mộc 8 tỷ người vào 15/11/2022. Theo báo cáo của Liên Hợp Quốc, việc thế giới đạt dấu mốc quan trọng này vừa là dịp để vui mừng, song cũng là dịp nhắc nhở về nhiệm vụ tìm ra giải pháp cho những thách thức mà nhân loại đang phải đối mặt. Theo đó, bài phân tích này hy vọng sẽ chỉ ra phần nào những thách thức mà vấn đề dân số đem lại trên thế giới. 


<b>Tạo một project mới với scrapy</b>


 Để sử dụng thư viện scrapy sau khi cài đặt xong, các bạn sẽ gọi câu lệnh như bên dưới để bắt đầu tạo một project mới với scrapy với tên gọi là "population".

In [ ]:
!scrapy startproject population

New Scrapy project 'population', using template directory 'c:\users\khoa\appdata\local\programs\python\python39\lib\site-packages\scrapy\templates\project', created in:
    c:\Users\Khoa\OneDrive\Desktop\20120120\KHDL\NMKHDL-20_21-Final-Project\population

You can start your first spider with:
    cd population
    scrapy genspider example example.com


In [ ]:
%cd population/population

c:\Users\MILU\Downloads\jupyter\population\population


- Tạo một file python "population_urls.py" để lấy đường link dẫn tới thông tin chi tiết các nước.

In [ ]:

url_spider_text='''\
import scrapy
class MySpider(scrapy.Spider):
    name = 'population_urls'
    #custom_settings = { 'CONCURRENT_REQUESTS': '1' }
    def start_requests(self):
        yield scrapy.Request('https://www.worldometers.info/world-population/population-by-country',self.parse)
    def parse(self, response):
        link_country= response.css("#example2 > tbody > tr > * > a::attr(href)").getall()
        yield dict.fromkeys(['population_urls'],link_country)
'''

if os.path.exists("./spiders/population_urls.py"):
    os.remove("./spiders/population_urls.py")
f=open("./spiders/population_urls.py",'w+')
f.write(url_spider_text)
f.close()

FileNotFoundError: [Errno 2] No such file or directory: './spiders/population_urls.py'

- Lưu thông tin vừa lấy được vào "population_urls.json"

In [ ]:

if os.path.exists("./dataset/population_urls.json"):
    os.remove("./dataset/population_urls.json")
!scrapy crawl population_urls -o dataset/population_urls.json

2022-12-09 15:29:03 [scrapy.utils.log] INFO: Scrapy 2.7.1 started (bot: population)
2022-12-09 15:29:03 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.12, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.0, Twisted 22.10.0, Python 3.7.8 (tags/v3.7.8:4b47a5b6ba, Jun 28 2020, 08:53:46) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 22.1.0 (OpenSSL 3.0.7 1 Nov 2022), cryptography 38.0.4, Platform Windows-10-10.0.19041-SP0
2022-12-09 15:29:03 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'population',
 'NEWSPIDER_MODULE': 'population.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['population.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'}
2022-12-09 15:29:03 [asyncio] DEBUG: Using selector: SelectSelector
2022-12-09 15:29:03 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2022-12-09 15:29:03 [scrapy.utils.log] DEBUG: Using asyncio event l

- Từ thông tin url các nước vừa lấy được, thu thập thông tin chi tiết các nước bằng file python "population.py".

In [ ]:

spider_text='''\
import scrapy
import json
import re
class collect_player_info(scrapy.Spider):
    name = 'population'
    url=[]
    def __init__(self):
        population_urls = [{'population_urls':''}]
        try:
            with open('./dataset/population_urls.json') as f:
                population_urls = json.load(f)
        except : print("File not found")
        self.url = population_urls[0]['population_urls']
        
    def start_requests(self):
        for i in self.url:
            yield scrapy.Request(f'https://www.worldometers.info{i}',self.parse)

    def parse(self, response):
        table= response.css('.table').get()
        table= scrapy.selector.Selector(text=table)
        dict_of_list=dict.fromkeys([''.join(table.xpath(f"//thead//th[{i+1}]/text()").getall()).replace(" ", "") for i in range(len(table.xpath("//thead//th").getall()))])
        dict_of_list['GlobalRank']=dict_of_list.pop(list(dict_of_list.keys())[-1])
        key_list=list(dict_of_list.keys())
        for i in range(len(key_list)):
            dict_of_list[key_list[i]]=table.xpath(f"//tbody//td[{i+1}]/node()").getall()+table.xpath(f"//thead//td[{i+1}]/node()").getall()
        dict_of_list['Population']=[re.search('(?<=\>)(.*?)(?=\<)', i)[0] for i in dict_of_list['Population']]
        dict_of_list['Country']=response.xpath("//div[@class='content']//li[@class='active']/text()").getall()*len(dict_of_list.pop('WorldPopulation'))
        dict_of_list['Continent']=response.xpath(f"//div[@class='content']/ul/li[4]/a/text()").getall()*len(dict_of_list['Country'])
        for i in [dict(zip(dict_of_list,t)) for t in zip(*dict_of_list.values())] :
            yield i
'''

if os.path.exists("./spiders/population.py"):
    os.remove("./spiders/population.py")
f=open("./spiders/population.py",'w+')
f.write(spider_text)
f.close()

- Nước Micronesia bị lỗi, thực hiện cào riêng nước đó.

In [ ]:

missing_spider_text='''\
import scrapy
import json
import re
class collect_player_info(scrapy.Spider):
    name = 'micronesia'
        
    def start_requests(self):
            i='/world-population/micronesia-population/'
            yield scrapy.Request(f'https://www.worldometers.info{i}',self.parse)

    def parse(self, response):
        table= response.css('body > div.container > div:nth-child(3) > div > div > div:nth-child(6) > table').get()
        table= scrapy.selector.Selector(text=table)
        dict_of_list=dict.fromkeys([''.join(table.xpath(f"//thead//th[{i+1}]/text()").getall()).replace(" ", "") for i in range(len(table.xpath("//thead//th").getall()))])
        dict_of_list['GlobalRank']=dict_of_list.pop(list(dict_of_list.keys())[-1])
        key_list=list(dict_of_list.keys())
        for i in range(len(key_list)):
            dict_of_list[key_list[i]]=table.xpath(f"//tbody//td[{i+1}]/node()").getall()+table.xpath(f"//thead//td[{i+1}]/node()").getall()
        dict_of_list["Country'sShareofWorldPop"]=dict_of_list.pop("Micronesia'sShareofWorldPop")
        dict_of_list['Population']=[re.search('(?<=\>)(.*?)(?=\<)', i)[0] for i in dict_of_list['Population']]
        dict_of_list['Country']=response.xpath("//div[@class='content']//li[@class='active']/text()").getall()*len(dict_of_list.pop('WorldPopulation'))
        dict_of_list['Continent']=response.xpath(f"//div[@class='content']/ul/li[4]/a/text()").getall()*len(dict_of_list['Country'])
        for i in [dict(zip(dict_of_list,t)) for t in zip(*dict_of_list.values())] :
            yield i




'''

if os.path.exists("./spiders/micronesia.py"):
    os.remove("./spiders/micronesia.py")
f=open("./spiders/micronesia.py",'w+')
f.write(missing_spider_text)
f.close()

- Lưu thông tin các nước vừa thu thập được bằng python vào file .json 

In [ ]:

if os.path.exists("./dataset/population.json"):
    os.remove("./dataset/population.json")
if os.path.exists("./dataset/micronesia.json"):
    os.remove("./dataset/micronesia.json")
!scrapy crawl population -o dataset/population.json
!scrapy crawl micronesia -o dataset/micronesia.json

2022-12-09 15:46:45 [scrapy.utils.log] INFO: Scrapy 2.7.1 started (bot: population)
2022-12-09 15:46:45 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.12, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.0, Twisted 22.10.0, Python 3.7.8 (tags/v3.7.8:4b47a5b6ba, Jun 28 2020, 08:53:46) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 22.1.0 (OpenSSL 3.0.7 1 Nov 2022), cryptography 38.0.4, Platform Windows-10-10.0.19041-SP0
2022-12-09 15:46:45 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'population',
 'NEWSPIDER_MODULE': 'population.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['population.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'}
2022-12-09 15:46:45 [asyncio] DEBUG: Using selector: SelectSelector
2022-12-09 15:46:45 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2022-12-09 15:46:45 [scrapy.utils.log] DEBUG: Using asyncio event l

- Đọc lại các file đã cào, kiểm tra dữ liệu đã thu thập đủ hay chưa.

In [ ]:
db=[]
with open('./dataset/population.json') as f:
        db = json.load(f)
df=pd.DataFrame(db)
with open('./dataset/micronesia.json') as f:
        db = json.load(f)
df=df.append(pd.DataFrame(db))
df=df.reset_index(drop=True)
total_country=['China', 'India', 'United States', 'Indonesia', 'Pakistan', 'Brazil', 'Nigeria', 'Bangladesh', 'Russia', 'Mexico', 'Japan', 'Ethiopia', 'Philippines', 'Egypt', 'Vietnam', 'DR Congo', 'Turkey', 'Iran', 'Germany', 'Thailand', 'United Kingdom', 'France', 'Italy', 'Tanzania', 'South Africa', 'Myanmar', 'Kenya', 'South Korea', 'Colombia', 'Spain', 'Uganda', 'Argentina', 'Algeria', 'Sudan', 'Ukraine', 'Iraq', 'Afghanistan', 'Poland', 'Canada', 'Morocco', 'Saudi Arabia', 'Uzbekistan', 'Peru', 'Angola', 'Malaysia', 'Mozambique', 'Ghana', 'Yemen', 'Nepal', 'Venezuela', 'Madagascar', 'Cameroon', "Côte d'Ivoire", 'North Korea', 'Australia', 'Niger', 'Taiwan', 'Sri Lanka', 'Burkina Faso', 'Mali', 'Romania', 'Malawi', 'Chile', 'Kazakhstan', 'Zambia', 'Guatemala', 'Ecuador', 'Syria', 'Netherlands', 'Senegal', 'Cambodia', 'Chad', 'Somalia', 'Zimbabwe', 'Guinea', 'Rwanda', 'Benin', 'Burundi', 'Tunisia', 'Bolivia', 'Belgium', 'Haiti', 'Cuba', 'South Sudan', 'Dominican Republic', 'Czech Republic (Czechia)', 'Greece', 'Jordan', 'Portugal', 'Azerbaijan', 'Sweden', 'Honduras', 'United Arab Emirates', 'Hungary', 'Tajikistan', 'Belarus', 'Austria', 'Papua New Guinea', 'Serbia', 'Israel', 'Switzerland', 'Togo', 'Sierra Leone', 'Hong Kong', 'Laos', 'Paraguay', 'Bulgaria', 'Libya', 'Lebanon', 'Nicaragua', 'Kyrgyzstan', 'El Salvador', 'Turkmenistan', 'Singapore', 'Denmark', 'Finland', 'Congo', 'Slovakia', 'Norway', 'Oman', 'State of Palestine', 'Costa Rica', 'Liberia', 'Ireland', 'Central African Republic', 'New Zealand', 'Mauritania', 'Panama', 'Kuwait', 'Croatia', 'Moldova', 'Georgia', 'Eritrea', 'Uruguay', 'Bosnia and Herzegovina', 'Mongolia', 'Armenia', 'Jamaica', 'Qatar', 'Albania', 'Puerto Rico', 'Lithuania', 'Namibia', 'Gambia', 'Botswana', 'Gabon', 'Lesotho', 'North Macedonia', 'Slovenia', 'Guinea-Bissau', 'Latvia', 'Bahrain', 'Equatorial Guinea', 'Trinidad and Tobago', 'Estonia', 'Timor-Leste', 'Mauritius', 'Cyprus', 'Eswatini', 'Djibouti', 'Fiji', 'Réunion', 'Comoros', 'Guyana', 'Bhutan', 'Solomon Islands', 'Macao', 'Montenegro', 'Luxembourg', 'Western Sahara', 'Suriname', 'Cabo Verde', 'Micronesia', 'Maldives', 'Malta', 'Brunei ', 'Guadeloupe', 'Belize', 'Bahamas', 'Martinique', 'Iceland', 'Vanuatu', 'French Guiana', 'Barbados', 'New Caledonia', 'French Polynesia', 'Mayotte', 'Sao Tome & Principe', 'Samoa', 'Saint Lucia', 'Channel Islands', 'Guam', 'Curaçao', 'Kiribati', 'Grenada', 'St. Vincent & Grenadines', 'Aruba', 'Tonga', 'U.S. Virgin Islands', 'Seychelles', 'Antigua and Barbuda', 'Isle of Man', 'Andorra', 'Dominica', 'Cayman Islands', 'Bermuda', 'Marshall Islands', 'Northern Mariana Islands', 'Greenland', 'American Samoa', 'Saint Kitts & Nevis', 'Faeroe Islands', 'Sint Maarten', 'Monaco', 'Turks and Caicos', 'Saint Martin', 'Liechtenstein', 'San Marino', 'Gibraltar', 'British Virgin Islands', 'Caribbean Netherlands', 'Palau', 'Cook Islands', 'Anguilla', 'Tuvalu', 'Wallis & Futuna', 'Nauru', 'Saint Barthelemy', 'Saint Helena', 'Saint Pierre & Miquelon', 'Montserrat', 'Falkland Islands', 'Niue', 'Tokelau', 'Holy See']

[i for i in total_country if i not in  df['Country'].unique()] == []

True

<h2><b>Tiền xử lý dữ liệu</b></h2>

- Kiểm tra kiểu dữ liệu của các cột.

In [ ]:
print(df.dtypes)    
df.head()

Year                        object
Population                  object
Yearly%Change               object
YearlyChange                object
Migrants(net)               object
MedianAge                   object
FertilityRate               object
Density(P/Km²)              object
UrbanPop%                   object
UrbanPopulation             object
Country'sShareofWorldPop    object
GlobalRank                  object
Country                     object
Continent                   object
dtype: object


,Year,Population,Yearly%Change,YearlyChange,Migrants(net),MedianAge,FertilityRate,Density(P/Km²),UrbanPop%,UrbanPopulation,Country'sShareofWorldPop,GlobalRank,Country,Continent
0,2020,"1,439,323,776",0.39 %,"5,540,090","-348,399",38.4,1.69,153,60.8 %,"875,075,919",18.47 %,1,China,Asia
1,2019,"1,433,783,686",0.43 %,"6,135,900","-348,399",37.0,1.65,153,59.7 %,"856,409,297",18.59 %,1,China,Asia
2,2018,"1,427,647,786",0.47 %,"6,625,995","-348,399",37.0,1.65,152,58.6 %,"837,022,095",18.71 %,1,China,Asia
3,2017,"1,421,021,791",0.49 %,"6,972,440","-348,399",37.0,1.65,151,57.5 %,"816,957,613",18.83 %,1,China,Asia
4,2016,"1,414,049,351",0.51 %,"7,201,481","-348,399",37.0,1.65,151,56.3 %,"796,289,491",18.94 %,1,China,Asia


- Sau khi đã kiểm tra dữ liệu khớp với dữ liệu thu thập được, chúng ta tiến hành thay đổi kiểu dữ liệu của cột thành dạng số thích hợp để phân tích.

In [ ]:
def dfStringToLargeInt(df,attr):
    df[attr]=[int(''.join(i.split(','))) if str(i)!='nan' and str(i)!='N.A.' else np.nan for i in df[attr]]
def dfStringToFloat(df,attr):
    df[attr]=[float(i) for i in df[attr]]
def dfStringRemovePercent(df,attr):
    df[attr]=[float(i.split('%')[0]) if i!='N.A.' else np.nan for i in df[attr]]
dfStringToLargeInt(df,'Year')
dfStringToLargeInt(df,'Population')
dfStringRemovePercent(df,'Yearly%Change')
dfStringToLargeInt(df,'YearlyChange')
dfStringToLargeInt(df,'Migrants(net)')
dfStringToFloat(df,'MedianAge')
dfStringToFloat(df,'FertilityRate')  
dfStringToLargeInt(df,'Density(P/Km²)')
dfStringRemovePercent(df,'UrbanPop%')
dfStringToLargeInt(df,'UrbanPopulation')
dfStringRemovePercent(df,"Country'sShareofWorldPop")
dfStringToLargeInt(df,'GlobalRank')

- Lấy các cột cần thiết, sắp xếp lại theo tên nước, năm thống kê.

In [ ]:
df.columns=['Year', 'Population', 'Yearly%Change', 'YearlyChange', 'Migrants(net)',
'MedianAge', 'FertilityRate', 'Density(P/Km2)', 'UrbanPop%',
       'UrbanPopulation', "Country%OfWorldPop", 'GlobalRank', 'Country','Continent']

In [ ]:
df=df.sort_values(by=['Country','Year'],ascending=True)
df=df.reset_index(drop=True)

- Chuyển đổi file dữ liệu .json sang .csv và lưu vào project chính để dễ tiếp cận hơn với dữ liệu.

In [ ]:
%cd ../../

c:\Users\MILU\Downloads\jupyter


In [ ]:
df.to_csv('./population.csv')

- Sau khi thực hiện thu thập dữ liệu hoàn tất, vì các file python và json trong thư mục population không cần thiết nữa. Ta xóa cây thư mục population.

In [ ]:
shutil.rmtree('./population')